## Generate HeFESTo lookup table for ASPEFT

In [ ]:
import os, sys
import numpy as np
from shutil import rmtree, copy
from matplotlib import pyplot as plt
from matplotlib import gridspec, cm

# directory to the aspect Lab
ASPECT_LAB_DIR = os.environ['ASPECT_LAB_DIR']
RESULT_DIR = os.path.join(ASPECT_LAB_DIR, 'results')

sys.path.append(os.path.join(ASPECT_LAB_DIR))
# import shilofue.PlotDepthAverage as PDAver
# import shilofue.FlowLaws as FlowLaws
# import shilofue.ParsePrm as ParsePrm
# import shilofue.ThermalModel as TModel
# import shilofue.Rheology as Rheology
import shilofue.PostHefesto as PostHefesto

# import utilities in subdirectiory
sys.path.append(os.path.join(ASPECT_LAB_DIR, 'utilities', "python_scripts"))
import Utilities

### Convert fort.56 file from HeFESTo to importable table to ASPECT

#### 1. Assign the path to the "fort.56" file below.

#### 2. Assign the path of the output.

For ASPECT to import the table, we need both the correct format of the table header and the two dimentional variables in the first, second columns.
To Do the conversion, the following flocks are needed.

First the P-T table.

In [ ]:
# P T table
input_file = "/home/lochy/ASPECT_PROJECT/HeFESTo_DIR/hefesto_pyrolite_P_T/output/fort.56"
output_file = "/home/lochy/ASPECT_PROJECT/HeFESTo_DIR/hefesto_pyrolite_P_T/output/pyrolite_PT.txt"

In [ ]:
# assert input file exists
assert(os.path.isfile(input_file))

if (os.path.isfile(output_file)):  # remove old files
    os.remove(output_file)

# read input
LookupTable = PostHefesto.LOOKUP_TABLE()
LookupTable.ReadRawFort56(input_file)

# process output
field_names = ['Pressure', 'Temperature', "Entropy", 'Density', 'Thermal_expansivity', 'Isobaric_heat_capacity', 'VP', 'VS', 'Enthalpy']
LookupTable.Process(field_names, output_file, interval1=1, interval2=1, file_type="structured", fix_coordinate_minor=True)

# assert something 
assert(os.path.isfile(output_file))

Then the P S table:

In [ ]:
# P S table
input_file = "/home/lochy/ASPECT_PROJECT/HeFESTo_DIR/hefesto_pyrolite_P_S/output/fort.56"
output_file = "/home/lochy/ASPECT_PROJECT/HeFESTo_DIR/hefesto_pyrolite_P_S/output/pyrolite_PS.txt"

In [ ]:
# read input
LookupTable = PostHefesto.LOOKUP_TABLE()
LookupTable.ReadRawFort56(input_file)

# process output
field_names = ['Pressure', 'Entropy', 'Temperature', 'Density', 'Thermal_expansivity', 'Isobaric_heat_capacity', 'VP', 'VS', 'Enthalpy']
LookupTable.Process(field_names, output_file, interval1=1, interval2=1, second_dimension="Entropy", file_type="structured", fix_coordinate_minor=True)

# assert something 
assert(os.path.isfile(output_file))